In [2]:

import pandas as pd

# F1 2021 UDP Specification

In [3]:
from ctypes import *


class Header(LittleEndianStructure):
    """
    Ctypes data structure for the header of a F1 2021 UDP packet
    https://github.com/raweceek-temeletry/f1-2021-udp
    """
    _pack_ = 1
    _fields_ = [
        ('m_packetFormat', c_uint16),     # 2021
        ('m_gameMajorVersion', c_uint8),  # Game major version - "X.00"
        ('m_gameMinorVersion', c_uint8),  # Game minor version - "1.XX"
        ('m_packetVersion', c_uint8),     # Version of this packet type, all start from 1
        ('m_packetId', c_uint8),          # Identifier for the packet type, see below
        ('m_sessionUID', c_uint64),       # Unique identifier for the session
        ('m_sessionTime', c_float),       # Session timestamp
        ('m_frameIdentifier', c_uint),    # Identifier for the frame the data was retrieved on
        ('m_playerCarIndex', c_uint8)     # Index of player's car in the array
    ]

# =============================================   0: MOTION    =====================================================================
class CarMotionData(LittleEndianStructure):
    """
    Ctypes data structure for a single car motion data

    The motion packet gives physics data for all the cars being driven. There is additional data for the car being
    driven with the goal of being able to drive a motion platform setup.

    Frequency: Rate as specified in menus
    """
    _pack_ = 1
    _fields_ = [
        ('m_worldPositionX', c_float),      # World space X position
        ('m_worldPositionY', c_float),      # World space Y position
        ('m_worldPositionZ', c_float),      # World space Z position
        ('m_worldVelocityX', c_float),      # Velocity in world space X
        ('m_worldVelocityY', c_float),      # Velocity in world space Y
        ('m_worldVelocityZ', c_float),      # Velocity in world space Z
        ('m_worldForwardDirX', c_int16),   # World space forward X direction (normalised)
        ('m_worldForwardDirY', c_int16),   # World space forward Y direction (normalised)
        ('m_worldForwardDirZ', c_int16),   # World space forward Z direction (normalised)
        ('m_worldRightDirX', c_int16),     # World space right X direction (normalised)
        ('m_worldRightDirY', c_int16),     # World space right Y direction (normalised)
        ('m_worldRightDirZ', c_int16),     # World space right Z direction (normalised)
        ('m_gForceLateral', c_float),       # Lateral G-Force component
        ('m_gForceLongitudinal', c_float),  # Longitudinal G-Force component
        ('m_gForceVertical', c_float),      # Vertical G-Force component
        ('m_yaw', c_float),                 # Yaw angle in radians
        ('m_pitch', c_float),               # Pitch angle in radians
        ('m_roll', c_float)                 # Roll angle in radians
    ]


class PacketMotionData(LittleEndianStructure):
    """
    Ctypes data structure for all cars motion data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                                # Header
        ('m_carMotionData', CarMotionData * 20),          # Data for all cars on track
        ('m_suspensionPosition', c_float * 4),      # Note: All wheel arrays have the following order:
        ('m_suspensionVelocity', c_float * 4),      # RL, RR, FL, FR
        ('m_suspensionAcceleration', c_float * 4),  # RL, RR, FL, FR
        ('m_wheelSpeed', c_float * 4),              # Speed of each wheel
        ('m_wheelSlip', c_float * 4),               # Slip ratio for each wheel
        ('m_localVelocityX', c_float),              # Velocity in local space
        ('m_localVelocityY', c_float),              # Velocity in local space
        ('m_localVelocityZ', c_float),              # Velocity in local space
        ('m_angularVelocityX', c_float),            # Angular velocity x-component
        ('m_angularVelocityY', c_float),            # Angular velocity y-component
        ('m_angularVelocityZ', c_float),            # Angular velocity z-component
        ('m_angularAccelerationX', c_float),        # Angular velocity x-component
        ('m_angularAccelerationY', c_float),        # Angular velocity y-component
        ('m_angularAccelerationZ', c_float),        # Angular velocity z-component
        ('m_frontWheelsAngle', c_float)             # Current front wheels angle in radians
    ]

# =============================================   1: SESSION    =====================================================================
class MarshalZone(LittleEndianStructure):
    """
    Ctypes data structure for the car data portion of a F1 2019 UDP packet
    """
    _pack_ = 1
    _fields_ = [
        ('m_zoneStart', c_float),  # Fraction (0..1) of way through the lap the marshal zone starts
        ('m_zoneFlag', c_int8)     # -1 = invalid/unknown, 0 = none, 1 = green, 2 = blue, 3 = yellow, 4 = red
    ]

class WeatherForecastSample(LittleEndianStructure):
    _pack = 1
    _fields = [

        ('m_sessionType', c_uint8),
                                             
                                                  # 0 = unknown, 1 = P1, 2 = P2, 3 = P3, 4 = Short P, 5 = Q1
                                                  # 6 = Q2, 7 = Q3, 8 = Short Q, 9 = OSQ, 10 = R, 11 = R2
                                                  # 12 = R3, 13 = Time Trialu
        ('m_timeOffset', c_int8),                 # Time in minutes the forecast is foru
        ('m_weather', c_int8),                    # Weather -0 = clear, 1 = light cloud, 2 = overcast# 3 = light rain, 4 = heavy rain, 5 = storm
        ('m_trackTemperature', c_int8),           # Track temp. in degrees Celsius
        ('m_trackTemperatureChange', c_int8),     # Track temp. change –0 = up, 1 = down, 2 = no change
        ('m_airTemperature', c_int8),             # Air temp. in degrees celsius
        ('m_airTemperatureChange', c_int8),       # Air temp. change –0 = up, 1 = down, 2 = no changeu
        ('m_rainPercentage', c_uint8)              # Rain percentage (0-100)
    ] 


class PacketSessionData(LittleEndianStructure):
    """
    Ctypes data structure for Session Data

    The session packet includes details about the current session in progress.

    Frequency: 2 per second
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                         # Header
        ('m_weather', c_uint8),              # Weather - 0 = clear, 1 = light cloud, 2 = overcast
                                                    # 3 = light rain, 4 = heavy rain, 5 = storm
        ('m_trackTemperature', c_int8),      # Track temp. in degrees celsius
        ('m_airTemperature', c_int8),        # Air temp. in degrees celsius
        ('m_totalLaps', c_uint8),            # Total number of laps in this race
        ('m_trackLength', c_uint16),         # Track length in metres
        ('m_sessionType', c_uint8),          # 0 = unknown, 1 = P1, 2 = P2, 3 = P3, 4 = Short P,  5 = Q1, 6 = Q2
                                                    # 7 = Q3, 8 = Short Q, 9 = OSQ, 10 = R, 11 = R2, 12 = Time Trial
        ('m_trackId', c_int8),               # -1 for unknown, 0-21 for tracks, see appendix
        ('m_formula', c_uint8),                  # Formula, 0 = F1 Modern, 1 = F1 Classic, 2 = F2, 3 = F1 Generic
        ('m_sessionTimeLeft', c_uint16),     # Time left in session in seconds
        ('m_sessionDuration', c_uint16),     # Session duration in seconds
        ('m_pitSpeedLimit', c_uint8),        # Pit speed limit in kilometres per hour
        ('m_gamePaused', c_uint8),           # Whether the game is paused
        ('m_isSpectating', c_uint8),         # Whether the player is spectating
        ('m_spectatorCarIndex', c_uint8),    # Index of the car being spectated
        ('m_sliProNativeSupport', c_uint8),  # SLI Pro support, 0 = inactive, 1 = active
        ('m_numMarshalZones', c_uint8),      # Number of marshal zones to follow
        ('m_marshalZones', MarshalZone * 21),       # List of marshal zones - max 21List of marshal zones - max 21
        ('m_safetyCarStatus', c_uint8),      # 0 = no safety car, 1 = full safety car, 2 = virtual safety car
        ('m_networkGame', c_uint8),          # 0 = offline, 1 = online  
        ('m_numWeatherForecastSamples', c_uint8),                     # Number of weather samples to follo
        ('m_weatherForecastSamples', WeatherForecastSample * 56) ,     # Array of weather forecast samples
        ('m_forecastAccuracy', c_uint8),                              # 0 =Perfect, 1 = Approximate
        ('m_aiDifficulty', c_uint8),                                  # AI Difficultyrating–0-110uint32m_seasonLinkIdentifier;      # Identifier for season -persists across savesuint32m_weekendLinkIdentifier;     # Identifier for weekend -persists across savesuint32m_sessionLinkIdentifier;     # Identifier for session -persists across saves
        ('m_pitStopWindowIdealLap', c_uint8),                         # Ideal lap to pit on for current strategy (player)
        ('m_pitStopWindowLatestLap', c_uint8),                        # Latest lap to pit on for current strategy (player)
        ('m_pitStopRejoinPosition', c_uint8),                         # Predicted position to rejoin at (player)
        ('m_steeringAssist', c_uint8),                                # 0 = off, 1 = on
        ('m_brakingAssist', c_uint8),                                 # 0 = off, 1 = low, 2 = medium, 3 = high
        ('m_gearboxAssist', c_uint8),                                 # 1 = manual, 2 = manual & suggested gear, 3 = auto
        ('m_pitAssist', c_uint8),                                     # 0 = off, 1 = on
        ('m_pitReleaseAssist', c_uint8),                              # 0 = off, 1 = on
        ('m_ERSAssist', c_uint8),                                     # 0 = off, 1 = on
        ('m_DRSAssist', c_uint8),                                     # 0 = off, 1 = on
        ('m_dynamicRacingLine', c_uint8),                             # 0 = off, 1 = corners only, 2 = full
        ('m_dynamicRacingLineType', c_uint8),                         # 0 = 2D, 1 = 3D
        ('m_gameMode', c_uint8),                                      # Game mode id -see appendix
        ('m_ruleSet', c_uint8),                                       # Ruleset -see appendixuint32    m_timeOfDay;                # Local time of day -minutes since midnight
        ('m_sessionLength', c_uint8),                                 # 0 = None, 2 = Very Short, 3 = Short, 4 = Medium#5 = Medium Long, 6 = Long, 7 = Full

    ]

# =============================================   2: LAP DATA    =====================================================================
class LapData(LittleEndianStructure):
    """
    The lap data packet gives details of all the cars in the session.

    Frequency: Rate as specified in menus
    """
    _pack_ = 1
    _fields_ = [
        ('m_lastLapTimeInMS', c_uint32),                #Last lap time in milliseconds
        ('m_currentLapTimeInMS', c_uint32),             #Current time around the lap in milliseconds
        ('m_sector1TimeInMS', c_uint16),                # Sector 1 time in milliseconds
        ('m_sector2TimeInMS', c_uint16),                # Sector 2 time in millisecondsfloat m_lapDistance;#Distance vehicle is around current lap in metres–could# be negative if line hasn’t been crossedyetfloat    m_totalDistance;# Total distance travelled in session in metres–could# be negative if line hasn’t been crossed yetfloat    m_safetyCarDelta;        # Delta in seconds for safety car
        ('m_carPosition', c_uint8 ),                    # Car race position
        ('m_currentLapNum', c_uint8 ),                  #Current lap number
        ('m_pitStatus', c_uint8 ),                      # 0 = none, 1 = pitting,2 = in pit area
        ('m_numPitStops', c_uint8 ),                    # Number ofpit stopstaken in this race
        ('m_sector', c_uint8 ),                         # 0 = sector1, 1 = sector2, 2 = sector3
        ('m_currentLapInvalid', c_uint8 ),              # Current lap invalid -0 = valid, 1= invalid
        ('m_penalties', c_uint8 ),                      # Accumulated time penalties in seconds to be added
        ('m_warnings', c_uint8 ),                       # Accumulated number of warnings issued
        ('m_numUnservedDriveThroughPens', c_uint8 ),    # Num drive through pens left to serve
        ('m_numUnservedStopGoPens', c_uint8 ),          # Num stop go pens left to serve
        ('m_gridPosition', c_uint8 ),                   # Grid position the vehicle started the racein
        ('m_driverStatus', c_uint8 ),                   # Status of driver -0 = in garage, 1 = flying lap# 2 = in lap, 3 = out lap, 4 = on track
        ('m_resultStatus', c_uint8 ),                   # Result status -0 = invalid, 1 = inactive, 2 = active#3 = finished, 4 = didnotfinish, 5= disqualified#6 = not classified, 7= retired
        ('m_pitLaneTimerActive', c_uint8 ),             # Pit lane timing, 0 = inactive, 1 = active
        ('m_pitLaneTimeInLaneInMS', c_uint16),          # If active, the current time spent in the pit lane in ms
        ('m_pitStopTimerInMS', c_uint16),               # Time of the actual pit stop in ms
        ('m_pitStopShouldServePen', c_uint8 ),          # Whether the car should serve a penalty at this stop
    ]


class PacketLapData(LittleEndianStructure):
    """
    Ctypes data structure for Lap data for all cars on track
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),               # Header
        ('m_lapData', LapData * 22),               # Lap data for all cars on track
        ('m_timeTrialPBCarIdx', c_uint8),         # Index of Personal Bestcar in time trial (255 if invalid)
        ('m_timeTrialRivalCarIdx', c_uint8),      # Index of Rival car in time trial (255 if invalid)
    ]


# =============================================    3: EVENT    =====================================================================
class EventDataDetails(LittleEndianStructure):
    """
    This packet gives details of events that happen during the course of a session.

    Frequency: When the event occurs
    """
    _pack_ = 1
    _fields_ = [

    ]


class PacketEventData(LittleEndianStructure):
    """
    This packet gives details of events that happen during the course of the race.

    Frequency: When the event occurs
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                         # Header
        ('m_eventStringCode', c_uint8 * 4),  # Event string code, see below
        ('m_eventDetails', EventDataDetails),       # Event details - should be interpreted differently for each type
    ]


# =============================================    4: PARTICIPANTS    =====================================================================
class ParticipantData(LittleEndianStructure):
    """
    This is a list of participants in the race. If the vehicle is controlled by AI, then the name will be the driver
    name. If this is a multiplayer game, the names will be the Steam Id on PC, or the LAN name if appropriate.
    On Xbox One, the names will always be the driver name, on PS4 the name will be the LAN name if playing a LAN game,
    otherwise it will be the driver name.

    Frequency: Every 5 seconds
    """
    _pack_ = 1
    _fields_ = [
        ('m_aiControlled', c_uint8),  # Whether the vehicle is AI (1) or Human (0) controlled
        ('m_driverId', c_uint8),      # Driver id - see appendix
        ('m_networkId', c_uint8),     # Network id –unique identifier for network players
        ('m_teamId', c_uint8),        # Team id - see appendix
        ('m_myTeam', c_int8),         # My team flag –1 = My Team, 0 = otherwise
        ('m_raceNumber', c_uint8),    # Race number of the car
        ('m_nationality', c_uint8),   # Nationality of the driver
        ('m_name', c_char * 48),      # Name of participant in UTF-8 format - null terminated
                                             # Will be truncated with (U+2026) if too long
        ('m_yourTelemetry', c_uint8)  # The player's UDP setting, 0 = restricted, 1 = public
    ]


class PacketParticipantsData(LittleEndianStructure):
    """
    Ctypes data structure for participants data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                           # Header
        ('m_numActiveCars', c_uint8),                # Number of cars in the data
        ('m_participants', ParticipantData * 20),
    ]


# =============================================    5: CAR SETUPS    =====================================================================
class CarSetupData(LittleEndianStructure):
    """
    This packet details the car setups for each vehicle in the session. Note that in multiplayer games, other player
    cars will appear as blank, you will only be able to see your car setup and AI cars.

    Frequency: Every 5 seconds
    """
    _pack_ = 1
    _fields_ = [
         ('m_frontWing', c_uint8),                 # Front wing aero
         ('m_rearWing', c_uint8),                  # Rear wing aero
         ('m_onThrottle', c_uint8),                # Differential adjustment on throttle (percentage)
         ('m_offThrottle', c_uint8),               # Differential adjustment off throttle (percentage)
         ('m_frontCamber', c_float),               # Front camber angle (suspension geometry)
         ('m_rearCamber', c_float),                # Rear camber angle (suspension geometry)
         ('m_frontToe', c_float),                  # Front toe angle (suspension geometry)
         ('m_rearToe', c_float),                   # Rear toe angle (suspension geometry)
         ('m_frontSuspension', c_uint8),           # Front suspension
         ('m_rearSuspension', c_uint8),            # Rear suspension
         ('m_frontAntiRollBar', c_uint8),          # Front anti-roll bar
         ('m_rearAntiRollBar', c_uint8),           # Front anti-roll bar
         ('m_frontSuspensionHeight', c_uint8),     # Front ride height
         ('m_rearSuspensionHeight', c_uint8),      # Rear ride height
         ('m_brakePressure', c_uint8),             # Brake pressure (percentage)
         ('m_brakeBias', c_uint8),                 # Brake bias (percentage)
         ('m_rearLeftTyrePressure', c_float),      # Rear left tyre pressure (PSI)
         ('m_rearRightTyrePressure', c_float),     # Rear right tyre pressure (PSI)
         ('m_frontLeftTyrePressure', c_float),     # Front left tyre pressure (PSI)
         ('m_frontRightTyrePressure', c_float),    # Front right tyre pressure (PSI)
         ('m_ballast', c_uint8),                   # Ballast
         ('m_fuelLoad', c_float),                  # Fuel load
    ]


class PacketCarSetupData(LittleEndianStructure):
    """
    Ctypes data structure for Cars setup
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),  # Header
        ('m_carSetups', CarSetupData * 22)
    ]

# =============================================    6: CAR TELEMETRY    =====================================================================
class CarTelemetryData(LittleEndianStructure):
    """
    This packet details telemetry for all the cars in the race. It details various values that would be recorded on the
    car such as speed, throttle application, DRS etc.

    Frequency: Rate as specified in menus
    """
    _pack_ = 1
    _fields_ = [
        ('m_speed', c_uint16),                              #  Speed of car in kilometres per hour
        ('m_throttle', c_float),                            #  Amount of throttle applied (0.0 to 1.0),
        ('m_steer', c_float ),                              #  Steering (-1.0 (full lock left), to 1.0 (full lock right),),
        ('m_brake', c_float ),                              #  Amount of brake applied (0.0 to 1.0),
        ('m_clutch', c_uint8),                              #  Amount of clutch applied (0 to 100),
        ('m_gear', c_int8   ),                              #  Gear selected (1-8, N=0, R=-1),
        ('m_engineRPM', c_uint16),                          #  Engine RPM
        ('m_drs', c_uint8   ),                              #  0 = off, 1 = on
        ('m_revLightsPercent', c_uint8),                    #  Rev lights indicator (percentage),
        ('m_revLightsBitValue', c_uint16),                  #  Rev lights (bit 0 = leftmost LED, bit 14 = rightmost LED),
        ('m_brakesTemperature', c_uint16*4),                # Brakes temperature (celsius),
        ('m_tyresSurfaceTemperature', c_uint8 *4),          # Tyres surface temperature (celsius),
        ('m_tyresInnerTemperature', c_uint8*4),             # Tyres inner temperature (celsius),
        ('m_engineTemperature', c_uint16),                  #  Engine temperature (celsius),
        ('m_tyresPressure', c_float *4),                    # Tyres pressure (PSI),
        ('m_surfaceType', c_uint8   *4),                    # Driving surface, see appendices
    ]


class PacketCarTelemetryData(LittleEndianStructure):
    """
    Ctypes data structure for Cars telemetry Data
    """
    _pack_ = 1
    _fields_ = [
      	('m_header', Header)	,                        # Header
        ('m_carTelemetryData', CarTelemetryData * 22) ,
        ('m_mfdPanelIndex', c_uint8),                    # Index of MFD panel open - 255 = MFD closed
                                                        # Single player, race – 0 = Car setup, 1 = Pits
                                                        # 2 = Damage, 3 =  Engine, 4 = Temperatures
                                                        # May vary depending on game mode
        ('m_mfdPanelIndexSecondaryPlayer', c_uint8),    # See above
        ('m_suggestedGear', c_int8 ),                   # Suggested gear for the player (1-8)
                                                        # 0 if no gear suggested
    ]

# =============================================    7: CAR STATUS    =====================================================================
class CarStatusData(LittleEndianStructure):
    """
    This packet details car statuses for all the cars in the race. It includes values such as the damage readings on the
    car.

    Frequency: 2 per second
    """
    _pack_ = 1
    _fields_ = [
        ('m_tractionControl', c_uint8),          # Traction control - 0 = off, 1 = medium, 2 = full
        ('m_antiLockBrakes', c_uint8),           # 0 (off) - 1 (on)
        ('m_fuelMix', c_uint8),                  # Fuel mix - 0 = lean, 1 = standard, 2 = rich, 3 = max
        ('m_frontBrakeBias', c_uint8),           # Front brake bias (percentage)
        ('m_pitLimiterStatus', c_uint8),         # Pit limiter status - 0 = off, 1 = on
        ('m_fuelInTank', c_float),               # Current fuel mass
        ('m_fuelCapacity', c_float),             # Fuel capacity
        ('m_fuelRemainingLaps', c_float),        # Fuel remaining in terms of laps (value on MFD)
        ('m_maxRPM', c_uint16),                  # Cars max RPM, point of rev limiter
        ('m_idleRPM', c_uint16),                 # Cars idle RPM
        ('m_maxGears', c_uint8),                 # Maximum number of gears
        ('m_drsAllowed', c_uint8),               # 0 = not allowed, 1 = allowed
        ('m_drsActivationDistance', c_uint16),   # 0 = DRS not available, non-zero - DRS will be available 
                                                # in [X] metres
        ('m_actualTyreCompound', c_uint8),	      # F1 Modern - 16 = C5, 17 = C4, 18 = C3, 19 = C2, 20 = C1# 7 = inter, 8 = wet
                                                # F1 Classic - 9 = dry, 10 = wet
                                                # # F2 – 11 = super soft, 12 = soft, 13 = medium, 14 = hard
                                                # # 15 = wet
        ('m_visualTyreCompound', c_uint8),       # F1 visual (can be different from actual compound) 
                                                # 16 = soft, 17 = medium, 18 = hard, 7 = inter, 8 = wet 
                                                # # F1 Classic – same as above 
                                                # # F2 ‘19, 15 = wet, 19 – super soft, 20 = soft 
                                                # # 21 = medium , 22 = hard
        ('m_tyresAgeLaps', c_uint8),             # Age in laps of the current set of tyres
        ('m_vehicleFiaFlags', c_int8),	          # -1 = invalid/unknown, 0 = none, 1 = green# 2 = blue, 3 = yellow, 4 = red
        ('m_ersStoreEnergy', c_float),           # ERS energy store in Joules
        ('m_ersDeployMode', c_uint8),            # ERS deployment mode, 0 = none, 1 = medium
                                                # 2 = hotlap, 3 = overtake
        ('m_ersHarvestedThisLapMGUK', c_float),  # ERS energy harvested this lap by MGU-K
        ('m_ersHarvestedThisLapMGUH', c_float),  # ERS energy harvested this lap by MGU-H
        ('m_ersDeployedThisLap', c_float),       # ERS energy deployed this lap
        ('m_networkPaused', c_uint8),            # Whether the car is paused in a network game
    ]


class PacketCarStatusData(LittleEndianStructure):
    """
    Ctypes data structure for cars status data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                              # Header
        ('m_carStatusData', CarStatusData * 22)
    ]

# =============================================    8: FINAL CLASSIFICATION    =====================================================================
class FinalClassificationData(LittleEndianStructure):
    """
    This packet details the final classification at the end of the race, 
    and the data will match with the post race results screen. 
    This is especially useful for multiplayer games where it is not always possible 
    to send lap times on the final frame because of network delay.

    Frequency: Once at the end of a race Size: 839 bytes Version: 1
    """

    _pack_ = 1
    _fields_ = [
        ('m_position', c_uint8),              # Finishing position
        ('m_numLaps', c_uint8),               # Number of laps completed
        ('m_gridPosition', c_uint8),          # Grid position of the car
        ('m_points', c_uint8),                # Number of points scored
        ('m_numPitStops', c_uint8),           # Number of pit stops made
        ('m_resultStatus', c_uint8),          # Result status - 0 = invalid, 1 = inactive, 2 = active// 3 = finished, 4 = didnotfinish, 5 = disqualified// 6 = not classified, 7 = retired
        ('m_bestLapTimeInMS', c_uint32),      # Best lap time of the session in milliseconds
        ('m_totalRaceTime', c_double),        # Total race time in seconds without penalties
        ('m_penaltiesTime', c_uint8),         # Total penalties accumulated in seconds
        ('m_numPenalties', c_uint8),          # Number of penalties applied to this driver
        ('m_numTyreStints', c_uint8),         # Number of tyres stints up to maximum
        ('m_tyreStintsActual', c_uint8 *8),   # Actual tyres used by this driver
        ('m_tyreStintsVisual', c_uint8 *8),   # Visual tyres used by this driver
    ]

class PacketFinalClassificationData(LittleEndianStructure):
    """
    Ctypes data structure for Cars telemetry Data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                                    # Header
        ('m_numCars', c_uint8),                                  # Number of cars in the final classification
        ('m_classificationData', FinalClassificationData * 22)
    ]

# =============================================    9: LOBBY INFO    =====================================================================
class LobbyInfoData(LittleEndianStructure):
    """
    This packet details the final classification at the end of the race, 
    and the data will match with the post race results screen. 
    This is especially useful for multiplayer games where it is not always possible 
    to send lap times on the final frame because of network delay.

    Frequency: Once at the end of a race Size: 839 bytes Version: 1
    """

    _pack_ = 1
    _fields_ = [
          ('m_aiControlled', c_uint8),   # Whether the vehicle is AI (1) or Human (0) controlled
          ('m_teamId', c_uint8),         # Team id - see appendix (255 if no team currently selected)
          ('m_nationality', c_uint8),    # Nationality of the driver
          ('m_name', c_char * 48), 		 # Name of participant in UTF-8 format – null terminated #
                                         # Will be truncated with ... (U+2026) if too long
          ('m_carNumber', c_uint8),      # Car number of the player
          ('m_readyStatus', c_uint8),    # 0 = not ready, 1 = ready, 2 = spectating
    ]

class PacketLobbyInfoData(LittleEndianStructure):
    """
    Ctypes data structure for Cars telemetry Data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                    # Header // Packet specific data
        ('m_numPlayers', c_uint8),               # Number of players in the lobby data
        ('m_lobbyPlayers', LobbyInfoData *22),
    ]


# =============================================    10: CAR DAMAGE    =====================================================================
class CarDamageData(LittleEndianStructure):
    """
    This packet details the final classification at the end of the race, 
    and the data will match with the post race results screen. 
    This is especially useful for multiplayer games where it is not always possible 
    to send lap times on the final frame because of network delay.

    Frequency: Once at the end of a race Size: 839 bytes Version: 1
    """

    _pack_ = 1
    _fields_ = [
         ('m_tyresWear', c_float *4),       # Tyre wear (percentage)
         ('m_tyresDamage', c_uint8 *4),    # Tyre damage (percentage)
         ('m_brakesDamage', c_uint8 *4),   # Brakes damage (percentage)
         ('m_frontLeftWingDamage', c_uint8),     # Front left wing damage (percentage)
         ('m_frontRightWingDamage', c_uint8),    # Front right wing damage (percentage)
         ('m_rearWingDamage', c_uint8),          # Rear wing damage (percentage)
         ('m_floorDamage', c_uint8),             # Floor damage (percentage)
         ('m_diffuserDamage', c_uint8),          # Diffuser damage (percentage)
         ('m_sidepodDamage', c_uint8),           # Sidepod damage (percentage)
         ('m_drsFault', c_uint8),                # Indicator for DRS fault, 0 = OK, 1 = fault
         ('m_gearBoxDamage', c_uint8),           # Gear box damage (percentage)
         ('m_engineDamage', c_uint8),            # Engine damage (percentage)
         ('m_engineMGUHWear', c_uint8),          # Engine wear MGU-H (percentage)
         ('m_engineESWear', c_uint8),            # Engine wear ES (percentage)
         ('m_engineCEWear', c_uint8),            # Engine wear CE (percentage)
         ('m_engineICEWear', c_uint8),           # Engine wear ICE (percentage)
         ('m_engineMGUKWear', c_uint8),          # Engine wear MGU-K (percentage)
         ('m_engineTCWear', c_uint8),            # Engine wear TC (percentage)
    ]

class PacketCarDamageData(LittleEndianStructure):
    """
    Ctypes data structure for Cars telemetry Data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                  # Header
        ('m_carDamageData', CarDamageData*22),
    ]



# =============================================    11: SESSION HISTORY    =====================================================================

class LapHistoryData(LittleEndianStructure):
    """
    Ctypes data structure for cars status data
    """
    _pack_ = 1
    _fields_ = [
         ('m_lapTimeInMS', c_uint32),           # Lap time in milliseconds
         ('m_sector1TimeInMS', c_uint16),       # Sector 1 time in milliseconds
         ('m_sector2TimeInMS', c_uint16),       # Sector 2 time in milliseconds
         ('m_sector3TimeInMS', c_uint16),       # Sector 3 time in milliseconds
         ('m_lapValidBitFlags', c_uint8),       #  0x01 bit set-lap valid,      0x02 bit set-sector 1 valid
                                                # 0x04 bit set-sector 2 valid, 0x08 bit set-sector 3 valid
    ]

class TyreStintHistoryData(LittleEndianStructure):
    """
    Ctypes data structure for Cars telemetry Data
    """
    _pack_ = 1
    _fields_ = [
        ('m_endLap', c_uint8),                # Lap the tyre usage ends on (255 of current tyre)
        ('m_tyreActualCompound', c_uint8),    # Actual tyres used by this driver
        ('m_tyreVisualCompound', c_uint8),    # Visual tyres used by this driver
    ]

class PacketSessionHistoryData(LittleEndianStructure):
    """
    Ctypes data structure for Cars telemetry Data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header) ,                  # Header
        ('m_carIdx', c_uint8) ,                              # Index of the car this lap data relates to
        ('m_numLaps', c_uint8) ,                             # Num laps in the data (including current partial lap)
        ('m_numTyreStints', c_uint8) ,                       # Number of tyre stints in the data
        ('m_bestLapTimeLapNum', c_uint8) ,                   # Lap the best lap time was achieved on
        ('m_bestSector1LapNum', c_uint8) ,                   # Lap the best Sector 1 time was achieved on
        ('m_bestSector2LapNum', c_uint8) ,                   # Lap the best Sector 2 time was achieved on
        ('m_bestSector3LapNum', c_uint8) ,                   # Lap the best Sector 3 time was achieved on
        ('m_lapHistoryData', LapHistoryData*100),            #  100 laps of data max
        ('m_tyreStintsHistoryData', TyreStintHistoryData*8)
    ]

In [25]:

def get_packet(d):
    m_header = Header.from_buffer_copy(d[0:23])
    packet = m_header.m_packetId
    # print(packet)

    if int(m_header.m_packetId) == 0:
        print('Motion')
        packet = PacketMotionData.from_buffer_copy(d[0:1464])
        theader = int(m_header.m_packetId)
        
    elif int(m_header.m_packetId) == 1:
        print('Session')
        packet = PacketLapData.from_buffer_copy(d[0:843])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 2:
        print('Lap Data')
        packet = PacketLapData.from_buffer_copy(d[0:970])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 3:
        print('Events')
        packet = PacketLapData.from_buffer_copy(d[0:36])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 4:
        print('Participants')
        packet = PacketLapData.from_buffer_copy(d[0:1257])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 5:
        print('Car Setups')
        packet = PacketLapData.from_buffer_copy(d[0:1102])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 6:
        print('Car Telemetry')
        packet = PacketLapData.from_buffer_copy(d[0:1347])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 7:
        print('Car Status')
        packet = PacketLapData.from_buffer_copy(d[0:1058])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)    
        
    elif int(m_header.m_packetId) == 8:
        print('Final Classification')
        packet = PacketLapData.from_buffer_copy(d[0:839])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 9:
        print('Lobby Info')
        packet = PacketLapData.from_buffer_copy(d[0:1191])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 10:
        print('Car Damage')
        packet = PacketLapData.from_buffer_copy(d[0:882])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    elif int(m_header.m_packetId) == 11:
        print('Session History')
        packet = PacketLapData.from_buffer_copy(d[0:1155])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        
    return (packet, theader, player)
        


In [ ]:
with open("data.txt", "rb") as f:
    data = f.readlines()

    f.close()

l = []


for i in range(len(data)):
    try:
        # print(i)

        data[i] = data[i].replace(b"b'", b"")
        data[i] = data[i].replace(b'"b""', b"")
        data[i] = data[i].replace(b"\r", b"")
        data[i] = data[i].replace(b"'\n", b"")
        data[i] = data[i].replace(b'"\n', b"")

        escaped_byte_literal = data[i]
        data[i] = escaped_byte_literal.decode('unicode_escape').encode('raw_unicode_escape')
        
        l.append(get_packet(data[i]))
    except:
        pass
    


In [29]:
telem = pd.DataFrame(l, columns=['PacketType', 'Packet', 'Player'])

In [31]:
telem[telem['Packet']==6]

,PacketType,Packet,Player
5,<__main__.PacketLapData object at 0x000001947D...,6,0
12,<__main__.PacketLapData object at 0x000001947D...,6,0
16,<__main__.PacketLapData object at 0x000001947D...,6,0
20,<__main__.PacketLapData object at 0x000001947D...,6,0
24,<__main__.PacketLapData object at 0x000001947D...,6,0
...,...,...,...
3535,<__main__.PacketLapData object at 0x000001947E...,6,0
4195,<__main__.PacketLapData object at 0x000001947E...,6,0
4479,<__main__.PacketLapData object at 0x000001947E...,6,0
4645,<__main__.PacketLapData object at 0x000001947E...,6,0


# F1 2019

In [70]:
import ctypes


class Header(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for the header of a F1 2019 UDP packet
    https://forums.codemasters.com/topic/38920-f1-2019-udp-specification/
    """
    _pack_ = 1
    _fields_ = [
        ('m_packetFormat', ctypes.c_uint16),     # 2019
        ('m_gameMajorVersion', ctypes.c_uint8),  # Game major version - "X.00"
        ('m_gameMinorVersion', ctypes.c_uint8),  # Game minor version - "1.XX"
        ('m_packetVersion', ctypes.c_uint8),     # Version of this packet type, all start from 1
        ('m_packetId', ctypes.c_uint8),          # Identifier for the packet type, see below
        ('m_sessionUID', ctypes.c_uint64),       # Unique identifier for the session
        ('m_sessionTime', ctypes.c_float),       # Session timestamp
        ('m_frameIdentifier', ctypes.c_uint),    # Identifier for the frame the data was retrieved on
        ('m_playerCarIndex', ctypes.c_uint8)     # Index of player's car in the array
    ]

# =================================================  0: MOTION   ==========================================================
class CarMotionData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for a single car motion data

    The motion packet gives physics data for all the cars being driven. There is additional data for the car being
    driven with the goal of being able to drive a motion platform setup.

    Frequency: Rate as specified in menus
    """
    _pack_ = 1
    _fields_ = [
        ('m_worldPositionX', ctypes.c_float),      # World space X position
        ('m_worldPositionY', ctypes.c_float),      # World space Y position
        ('m_worldPositionZ', ctypes.c_float),      # World space Z position
        ('m_worldVelocityX', ctypes.c_float),      # Velocity in world space X
        ('m_worldVelocityY', ctypes.c_float),      # Velocity in world space Y
        ('m_worldVelocityZ', ctypes.c_float),      # Velocity in world space Z
        ('m_worldForwardDirX', ctypes.c_int16),   # World space forward X direction (normalised)
        ('m_worldForwardDirY', ctypes.c_int16),   # World space forward Y direction (normalised)
        ('m_worldForwardDirZ', ctypes.c_int16),   # World space forward Z direction (normalised)
        ('m_worldRightDirX', ctypes.c_int16),     # World space right X direction (normalised)
        ('m_worldRightDirY', ctypes.c_int16),     # World space right Y direction (normalised)
        ('m_worldRightDirZ', ctypes.c_int16),     # World space right Z direction (normalised)
        ('m_gForceLateral', ctypes.c_float),       # Lateral G-Force component
        ('m_gForceLongitudinal', ctypes.c_float),  # Longitudinal G-Force component
        ('m_gForceVertical', ctypes.c_float),      # Vertical G-Force component
        ('m_yaw', ctypes.c_float),                 # Yaw angle in radians
        ('m_pitch', ctypes.c_float),               # Pitch angle in radians
        ('m_roll', ctypes.c_float)                 # Roll angle in radians
    ]


class PacketMotionData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for all cars motion data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                                # Header
        ('m_carMotionData', CarMotionData * 20),          # Data for all cars on track
        ('m_suspensionPosition', ctypes.c_float * 4),      # Note: All wheel arrays have the following order:
        ('m_suspensionVelocity', ctypes.c_float * 4),      # RL, RR, FL, FR
        ('m_suspensionAcceleration', ctypes.c_float * 4),  # RL, RR, FL, FR
        ('m_wheelSpeed', ctypes.c_float * 4),              # Speed of each wheel
        ('m_wheelSlip', ctypes.c_float * 4),               # Slip ratio for each wheel
        ('m_localVelocityX', ctypes.c_float),              # Velocity in local space
        ('m_localVelocityY', ctypes.c_float),              # Velocity in local space
        ('m_localVelocityZ', ctypes.c_float),              # Velocity in local space
        ('m_angularVelocityX', ctypes.c_float),            # Angular velocity x-component
        ('m_angularVelocityY', ctypes.c_float),            # Angular velocity y-component
        ('m_angularVelocityZ', ctypes.c_float),            # Angular velocity z-component
        ('m_angularAccelerationX', ctypes.c_float),        # Angular velocity x-component
        ('m_angularAccelerationY', ctypes.c_float),        # Angular velocity y-component
        ('m_angularAccelerationZ', ctypes.c_float),        # Angular velocity z-component
        ('m_frontWheelsAngle', ctypes.c_float)             # Current front wheels angle in radians
    ]


class MarshalZone(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for the car data portion of a F1 2019 UDP packet
    """
    _pack_ = 1
    _fields_ = [
        ('m_zoneStart', ctypes.c_float),  # Fraction (0..1) of way through the lap the marshal zone starts
        ('m_zoneFlag', ctypes.c_int8)     # -1 = invalid/unknown, 0 = none, 1 = green, 2 = blue, 3 = yellow, 4 = red
    ]


class PacketSessionData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for Session Data

    The session packet includes details about the current session in progress.

    Frequency: 2 per second
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                         # Header
        ('m_weather', ctypes.c_uint8),              # Weather - 0 = clear, 1 = light cloud, 2 = overcast
                                                    # 3 = light rain, 4 = heavy rain, 5 = storm
        ('m_trackTemperature', ctypes.c_int8),      # Track temp. in degrees celsius
        ('m_airTemperature', ctypes.c_int8),        # Air temp. in degrees celsius
        ('m_totalLaps', ctypes.c_uint8),            # Total number of laps in this race
        ('m_trackLength', ctypes.c_uint16),         # Track length in metres
        ('m_sessionType', ctypes.c_uint8),          # 0 = unknown, 1 = P1, 2 = P2, 3 = P3, 4 = Short P,  5 = Q1, 6 = Q2
                                                    # 7 = Q3, 8 = Short Q, 9 = OSQ, 10 = R, 11 = R2, 12 = Time Trial
        ('m_trackId', ctypes.c_int8),               # -1 for unknown, 0-21 for tracks, see appendix
        ('m_formula', ctypes.c_uint8),                  # Formula, 0 = F1 Modern, 1 = F1 Classic, 2 = F2, 3 = F1 Generic
        ('m_sessionTimeLeft', ctypes.c_uint16),     # Time left in session in seconds
        ('m_sessionDuration', ctypes.c_uint16),     # Session duration in seconds
        ('m_pitSpeedLimit', ctypes.c_uint8),        # Pit speed limit in kilometres per hour
        ('m_gamePaused', ctypes.c_uint8),           # Whether the game is paused
        ('m_isSpectating', ctypes.c_uint8),         # Whether the player is spectating
        ('m_spectatorCarIndex', ctypes.c_uint8),    # Index of the car being spectated
        ('m_sliProNativeSupport', ctypes.c_uint8),  # SLI Pro support, 0 = inactive, 1 = active
        ('m_numMarshalZones', ctypes.c_uint8),      # Number of marshal zones to follow
        ('m_marshalZones', MarshalZone * 21),       # List of marshal zones - max 21List of marshal zones - max 21
        ('m_safetyCarStatus', ctypes.c_uint8),      # 0 = no safety car, 1 = full safety car, 2 = virtual safety car
        ('m_networkGame', ctypes.c_uint8),          # 0 = offline, 1 = online
    ]


class LapData(ctypes.LittleEndianStructure):
    """
    The lap data packet gives details of all the cars in the session.

    Frequency: Rate as specified in menus
    """
    _pack_ = 1
    _fields_ = [
        ('m_lastLapTime', ctypes.c_float),        # Last lap time in seconds
        ('m_currentLapTime', ctypes.c_float),     # Current time around the lap in seconds
        ('m_bestLapTime', ctypes.c_float),        # Best lap time of the session in seconds
        ('m_sector1Time', ctypes.c_float),        # Sector 1 time in seconds
        ('m_sector2Time', ctypes.c_float),        # Sector 2 time in seconds
        ('m_lapDistance', ctypes.c_float),        # Distance vehicle is around current lap in metres - could
                                                  # be negative if line hasn't been crossed yet
        ('m_totalDistance', ctypes.c_float),      # Total distance travelled in session in metres - could
                                                  # be negative if line hasn't been crossed yet
        ('m_safetyCarDelta', ctypes.c_float),     # Delta in seconds for safety car
        ('m_carPosition', ctypes.c_uint8),        # Car race position
        ('m_currentLapNum', ctypes.c_uint8),      # Current lap number
        ('m_pitStatus', ctypes.c_uint8),          # 0 = none, 1 = pitting, 2 = in pit area
        ('m_sector', ctypes.c_uint8),             # 0 = sector1, 1 = sector2, 2 = sector3
        ('m_currentLapInvalid', ctypes.c_uint8),  # Current lap invalid - 0 = valid, 1 = invalid
        ('m_penalties', ctypes.c_uint8),          # Accumulated time penalties in seconds to be added
        ('m_gridPosition', ctypes.c_uint8),       # Grid position the vehicle started the race in
        ('m_driverStatus', ctypes.c_uint8),       # Status of driver - 0 = in garage, 1 = flying lap, 2 = in lap
                                                  # 3 = out lap, 4 = on track
        ('m_resultStatus', ctypes.c_uint8)        # Result status - 0 = invalid, 1 = inactive, 2 = active, 3 = finished
                                                  # 4 = disqualified, 5 = not classified, 6 = retired
    ]


class PacketLapData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for Lap data for all cars on track
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),          # Header
        ('m_lapsData', LapData * 20)  # Lap data for all cars on track
    ]

class EventDataDetails(ctypes.LittleEndianStructure):
    """
    This packet gives details of events that happen during the course of a session.

    Frequency: When the event occurs
    """
    _pack_ = 1
    _fields_ = [

    ]


class PacketEventData(ctypes.LittleEndianStructure):
    """
    This packet gives details of events that happen during the course of the race.

    Frequency: When the event occurs
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                         # Header
        ('m_eventStringCode', ctypes.c_uint8 * 4),  # Event string code, see below
        ('m_eventDetails', EventDataDetails),       # Event details - should be interpreted differently for each type
    ]


class ParticipantData(ctypes.LittleEndianStructure):
    """
    This is a list of participants in the race. If the vehicle is controlled by AI, then the name will be the driver
    name. If this is a multiplayer game, the names will be the Steam Id on PC, or the LAN name if appropriate.
    On Xbox One, the names will always be the driver name, on PS4 the name will be the LAN name if playing a LAN game,
    otherwise it will be the driver name.

    Frequency: Every 5 seconds
    """
    _pack_ = 1
    _fields_ = [
        ('m_aiControlled', ctypes.c_uint8),  # Whether the vehicle is AI (1) or Human (0) controlled
        ('m_driverId', ctypes.c_uint8),      # Driver id - see appendix
        ('m_teamId', ctypes.c_uint8),        # Team id - see appendix
        ('m_raceNumber', ctypes.c_uint8),    # Race number of the car
        ('m_nationality', ctypes.c_uint8),   # Nationality of the driver
        ('m_name', ctypes.c_char * 48),      # Name of participant in UTF-8 format - null terminated
                                             # Will be truncated with (U+2026) if too long
        ('m_yourTelemetry', ctypes.c_uint8)  # The player's UDP setting, 0 = restricted, 1 = public
    ]


class PacketParticipantsData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for participants data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                           # Header
        ('m_numActiveCars', ctypes.c_uint8),                # Number of cars in the data
        ('m_participants', ParticipantData * 20),
    ]


class CarSetupData(ctypes.LittleEndianStructure):
    """
    This packet details the car setups for each vehicle in the session. Note that in multiplayer games, other player
    cars will appear as blank, you will only be able to see your car setup and AI cars.

    Frequency: Every 5 seconds
    """
    _pack_ = 1
    _fields_ = [
        ('m_frontWing', ctypes.c_uint8),              # Front wing aero
        ('m_rearWing', ctypes.c_uint8),               # Rear wing aero
        ('m_onThrottle', ctypes.c_uint8),             # Differential adjustment on throttle (percentage)
        ('m_offThrottle', ctypes.c_uint8),            # Differential adjustment off throttle (percentage)
        ('m_frontCamber', ctypes.c_float),            # Front camber angle (suspension geometry)
        ('m_rearCamber', ctypes.c_float),             # Rear camber angle (suspension geometry)
        ('m_frontToe', ctypes.c_float),               # Front toe angle (suspension geometry)
        ('m_rearToe', ctypes.c_uint8),                # Rear toe angle (suspension geometry)
        ('m_frontSuspension', ctypes.c_uint8),        # Front suspension
        ('m_rearSuspension', ctypes.c_uint8),         # Rear suspension
        ('m_frontAntiRollBar', ctypes.c_uint8),       # Front anti-roll bar
        ('m_rearAntiRollBar', ctypes.c_uint8),        # Front anti-roll bar
        ('m_frontSuspensionHeight', ctypes.c_uint8),  # Front ride height
        ('m_rearSuspensionHeight', ctypes.c_uint8),   # Rear ride height
        ('m_brakePressure', ctypes.c_uint8),          # Brake pressure (percentage)
        ('m_brakeBias', ctypes.c_uint8),              # Brake bias (percentage)
        ('m_frontTyrePressure', ctypes.c_float),      # Front tyre pressure (PSI)
        ('m_rearTyrePressure', ctypes.c_float),       # Rear tyre pressure (PSI)
        ('m_ballast', ctypes.c_uint8),                # Ballast
        ('m_fuelLoad', ctypes.c_float),               # Fuel load
    ]


class PacketCarSetupData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for Cars setup
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),  # Header
        ('m_carSetups', CarSetupData * 20)
    ]


class CarTelemetryData(ctypes.LittleEndianStructure):
    """
    This packet details telemetry for all the cars in the race. It details various values that would be recorded on the
    car such as speed, throttle application, DRS etc.

    Frequency: Rate as specified in menus
    """
    _pack_ = 1
    _fields_ = [
        ('m_speed', ctypes.c_uint16),                        # Speed of car in kilometres per hour
        ('m_throttle', ctypes.c_float),                      # Amount of throttle applied (0 to 100)
        ('m_steer', ctypes.c_float),                         # Steering (-100 (full lock left) to 100 (full lock right))
        ('m_brake', ctypes.c_float),                         # Amount of brake applied (0 to 100)
        ('m_clutch', ctypes.c_uint8),                        # Amount of clutch applied (0 to 100)
        ('m_gear', ctypes.c_int8),                           # Gear selected (1-8, N=0, R=-1)
        ('m_engineRPM', ctypes.c_uint16),                      # Engine RPM
        ('m_drs', ctypes.c_uint8),                           # 0 = off, 1 = on
        ('m_revLightsPercent', ctypes.c_uint8),              # Rev lights indicator (percentage)
        ('m_brakesTemperature', ctypes.c_uint16 * 4),        # Brakes temperature (celsius)
        ('m_tyresSurfaceTemperature', ctypes.c_uint16 * 4),    # Tyres surface temperature (celsius)
        ('m_tyresInnerTemperature', ctypes.c_uint16 * 4),    # Tyres inner temperature (celsius)
        ('m_engineTemperature', ctypes.c_uint16),            # Engine temperature (celsius)
        ('m_tyresPressure', ctypes.c_float * 4),             # Tyres pressure (PSI)
        ('m_surfaceType', ctypes.c_uint8 * 4),               # Driving surface, see appendices
    ]


class PacketCarTelemetryData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for Cars telemetry Data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                              # Header
        ('m_carTelemetryData', CarTelemetryData * 20),
        ('m_buttonStatus', ctypes.c_uint32)                # Bit flags specifying which buttons are being pressed
                                                         # currently - see appendices
    ]


class CarStatusData(ctypes.LittleEndianStructure):
    """
    This packet details car statuses for all the cars in the race. It includes values such as the damage readings on the
    car.

    Frequency: 2 per second
    """
    _pack_ = 1
    _fields_ = [
        ('m_tractionControl', ctypes.c_uint8),          # 0 (off) - 2 (high)
        ('m_antiLockBrakes', ctypes.c_uint8),           # 0 (off) - 1 (on)
        ('m_fuelMix', ctypes.c_uint8),                  # Fuel mix - 0 = lean, 1 = standard, 2 = rich, 3 = max
        ('m_frontBrakeBias', ctypes.c_uint8),           # Front brake bias (percentage)
        ('m_pitLimiterStatus', ctypes.c_uint8),         # Pit limiter status - 0 = off, 1 = on
        ('m_fuelInTank', ctypes.c_float),               # Current fuel mass
        ('m_fuelCapacity', ctypes.c_float),             # Fuel capacity
        ('m_fuelRemainingLaps', ctypes.c_float),        # Fuel remaining in terms of laps (value on MFD)
        ('m_maxRPM', ctypes.c_uint16),                    # Cars max RPM, point of rev limiter
        ('m_idleRPM', ctypes.c_uint16),                   # Cars idle RPM
        ('m_maxGears', ctypes.c_uint8),                 # Maximum number of gears
        ('m_drsAllowed', ctypes.c_uint8),               # 0 = not allowed, 1 = allowed, -1 = unknown
        ('m_tyresWear', ctypes.c_uint8 * 4),            # Tyre wear percentage
        ('m_actualTyreCompound', ctypes.c_uint8),       # F1 Modern - 16 = C5, 17 = C4, 18 = C3, 19 = C2, 20 = C1
   					                                    # 7 = inter, 8 = wet
   					                                    # F1 Classic - 9 = dry, 10 = wet
   					                                    # F2 - 11 = super soft, 12 = soft, 13 = medium, 14 = hard
   					                                    # 15 = wet
        ('m_tyreVisualCompound', ctypes.c_uint8),       # F1 visual (can be different from actual compound
                                                        # 16 = soft, 17 = medium, 18 = hard, 7 = inter, 8 = wet
                                                        # F1 Classic - same as above
                                                        # F2 - same as above
        ('m_tyresDamage', ctypes.c_uint8 * 4),          # Tyre damage (percentage)
        ('m_frontLeftWingDamage', ctypes.c_uint8),      # Front left wing damage (percentage)
        ('m_frontRightWingDamage', ctypes.c_uint8),     # Front right wing damage (percentage)
        ('m_rearWingDamage', ctypes.c_uint8),           # Rear wing damage (percentage)
        ('m_engineDamage', ctypes.c_uint8),             # Engine damage (percentage)
        ('m_gearBoxDamage', ctypes.c_uint8),            # Gear box damage (percentage)
        ('m_vehicleFiaFlags', ctypes.c_int8),           # -1 = invalid/unknown, 0 = none, 1 = green, 2 = blue
                                                        #  3 = yellow, 4 = red
        ('m_ersStoreEnergy', ctypes.c_float),           # ERS energy store in Joules
        ('m_ersDeployMode', ctypes.c_uint8),            # ERS deployment mode, 0 = none, 1 = low, 2 = medium, 3 = high
                                                        #  4 = overtake, 5 = hotlap
        ('m_ersHarvestedThisLapMGUK', ctypes.c_float),  # ERS energy harvested this lap by MGU-K
        ('m_ersHarvestedThisLapMGUH', ctypes.c_float),  # ERS energy harvested this lap by MGU-H
        ('m_ersDeployedThisLap', ctypes.c_float),       # ERS energy deployed this lap
    ]


class PacketCarStatusData(ctypes.LittleEndianStructure):
    """
    Ctypes data structure for cars status data
    """
    _pack_ = 1
    _fields_ = [
        ('m_header', Header),                              # Header
        ('m_carStatusData', CarStatusData * 20)
    ]

In [72]:

def get_packet(data):
    m_header = Header.from_buffer_copy(data[0:23])
    mad = 0
    player = 0
    if int(m_header.m_packetId) == 0:
        packet = PacketMotionData.from_buffer_copy(data[0:1343])
        theader = int(m_header.m_packetId)

    elif int(m_header.m_packetId) == 1:
        packet = PacketSessionData.from_buffer_copy(data[0:149])
        theader = int(m_header.m_packetId)
        
        print(packet.m_sessionTimeLeft)
        
        # ('m_sessionTimeLeft', ctypes.c_uint16),     # Time left in session in seconds
        # ('m_sessionDuration', ctypes.c_uint16),     # Session duration in seconds
        # ('m_pitSpeedLimit', ctypes.c_uint8),        # Pit speed limit in kilometres per hour
        # ('m_gamePaused', ctypes.c_uint8),           # Whether the game is paused
        # ('m_isSpectating', ctypes.c_uint8),         # Whether the player is spectating
        # ('m_spectatorCarIndex', ctypes.c_uint8),    # Index of the car being spectated

    elif int(m_header.m_packetId) == 2:
        packet = PacketLapData.from_buffer_copy(data[0:843])
        theader = int(m_header.m_packetId)
        player = int(m_header.m_playerCarIndex)
        #mad = PacketLapData.from_buffer_copy(data[0:843])

    elif int(m_header.m_packetId) == 3:
        packet = PacketEventData.from_buffer_copy(data[0:32])
        theader = int(m_header.m_packetId)

    elif int(m_header.m_packetId) == 4:
        packet = PacketParticipantsData.from_buffer_copy(data[0:1104])
        theader = int(m_header.m_packetId)
        mad = PacketParticipantsData.from_buffer_copy(data[0:1104])

    elif int(m_header.m_packetId) == 5:
        packet = PacketCarSetupData.from_buffer_copy(data[0:843])
        theader = int(m_header.m_packetId)

    elif int(m_header.m_packetId) == 6:
        packet = PacketCarTelemetryData.from_buffer_copy(data[0:1347])
        theader = int(m_header.m_packetId)

    elif int(m_header.m_packetId) == 7:
        packet = PacketCarStatusData.from_buffer_copy(data[0:1143])
        theader = int(m_header.m_packetId)

    return packet, player, m_header, theader
print(packet)
print(m_header)
print(theader)
print(player)

6
0


In [73]:
with open("data.txt", "rb") as f:
    data = f.readlines()

    f.close()

l = []


for i in range(len(data)):
    try:
        # print(i)

        data[i] = data[i].replace(b"b'", b"")
        data[i] = data[i].replace(b'"b""', b"")
        data[i] = data[i].replace(b"\r", b"")
        data[i] = data[i].replace(b"'\n", b"")
        data[i] = data[i].replace(b'"\n', b"")

        escaped_byte_literal = data[i]
        data[i] = escaped_byte_literal.decode('unicode_escape').encode('raw_unicode_escape')
        
        l.append(get_packet(data[i]))
    except:
        pass
    
data = pd.DataFrame(l, columns=['packet', '0', 'header', 'packet type'])
data[data['packet type']==6]['packet'][72]

C:\Users\jules\AppData\Local\Temp\ipykernel_30864\1420173269.py:12: DeprecationWarning: invalid escape sequence '\q'
  data[i] = escaped_byte_literal.decode('unicode_escape').encode('raw_unicode_escape')
C:\Users\jules\AppData\Local\Temp\ipykernel_30864\1420173269.py:12: DeprecationWarning: invalid escape sequence '\'
  data[i] = escaped_byte_literal.decode('unicode_escape').encode('raw_unicode_escape')
C:\Users\jules\AppData\Local\Temp\ipykernel_30864\1420173269.py:12: DeprecationWarning: invalid escape sequence '\'
  data[i] = escaped_byte_literal.decode('unicode_escape').encode('raw_unicode_escape')
C:\Users\jules\AppData\Local\Temp\ipykernel_30864\1420173269.py:12: DeprecationWarning: invalid escape sequence '\¶'
  data[i] = escaped_byte_literal.decode('unicode_escape').encode('raw_unicode_escape')
C:\Users\jules\AppData\Local\Temp\ipykernel_30864\1420173269.py:12: DeprecationWarning: invalid escape sequence '\¬'
  data[i] = escaped_byte_literal.decode('unicode_escape').encode('r